In [1]:
__author__ = "Gareth Murphy"
__copyright__ = "Copyright 2016, DTU Space/ESA/ASDC"
__credits__ = ["Gareth Murphy"]
__license__ = "ESA Software Community License"
__version__ = "1.0.1"
__maintainer__ = "Gareth Murphy"
__email__ = "gmurphy@space.dtu.dk"
__status__ = "Beta"

%matplotlib inline

# Note: pandas data format is column major 



import pandas 
import numpy as np
import matplotlib.pyplot as plt
from  matplotlib.table import Table

def main():
    sz=4

    #checkerboard_table(dmdata2)
    
    
    n=16
    # setup pixel data

    pixelnum=np.linspace(1,n*n,n*n, dtype=np.int16)
    print (pixelnum.size)
    channelnum= np.array([1, 2, 3])
    
    
    values=np.reshape(pixelnum, (n, n))
    values=np.transpose(values)
    index=np.linspace(1,n,n)
    pixeldata=pandas.DataFrame(values, index=index)

    
    # setup channel data
    asic_c1= np.array([ 89, 87, 84, 78, 94, 85, 74, 68, 
                       61, 55, 49, 39, 42, 41, 38, 34, 
                       88, 86, 82, 80, 92, 83, 72, 66, 
                       63, 57, 51, 35, 47, 45, 40, 36, 
                       91, 97,103, 76, 96, 90, 81, 70, 
                       64, 59, 53, 44, 37, 43, 26, 30, 
                       93, 99,105, 98,104, 77, 71, 65, 
                       62, 56, 50, 21, 27, 31, 24, 32, 
                       95,101,107,100,106, 75, 69, 67, 
                       60, 54, 48, 33, 25, 29, 22, 28,
                       109,121,115,127,102, 79, 73,108, 
                        58, 52, 46,  5, 10,  6, 12, 20,
                       113,125,119,128,122,118,114,110, 
                       19, 13, 11,  7,  1,  4, 14, 18,
                       117,111,123,126,124,120,116,112, 
                       23, 17, 15,  9,  3,  8,  2, 16])
    
    asic_c2= np.array([ 20, 14,  4, 10,  7, 11, 15, 19,
                       110,114,118,124,127,115,121,109, 
                       18, 12,  6,  1,  5,  9, 13, 17,
                       108,112,116,122,128,119,125,113, 
                       16,  2,  8,  3, 33, 50, 56, 23, 
                       71, 77,104,120,126,123,111,117, 
                       28, 22, 27, 21, 48, 54, 60, 62, 
                       69, 75,106,102, 98,105,101, 95, 
                       32, 26, 29, 25, 46, 52, 58, 64, 
                       67, 73, 79,100,107,103, 93, 91, 
                       30, 24, 31, 43, 37, 44, 53, 59,
                       65, 70, 81, 90, 96, 76, 99, 97,
                       34, 40, 45, 47, 35, 51, 57, 63, 
                       66, 72, 83, 92, 80, 82, 86, 88, 
                       36, 38, 41, 42, 39, 49, 55, 61, 
                       68, 74, 85, 94, 78, 84, 87, 89])

    
    channelnum=np.concatenate((asic_c1, asic_c2), axis=0) 
    
    print (asic_c1.size)
    
    channeldata=np.reshape(channelnum, (n, n))
    channeldata=np.transpose(channeldata)
    channeldata=np.flipud(channeldata)
    index=np.linspace(1,n,n)
    channeldatadf=pandas.DataFrame(channeldata, index=index)
    
    #checkerboard_table(channeldatadf)
    #checkerboard_table(pixeldata)

    
    pczt=setupczt(pixeldata)
    asic_channel_numbers=setupczt(channeldata)

    index=np.linspace(1,n*n/2,n*n/2)
    
    #  setting up dau and asic numbers
    dau= np.array([ [2, 3], [1,4]] )

    asicno = np.array([[1,2],[1,2]])
    print dau

    asicno= asicno.repeat(8, axis=0).repeat(8, axis=1)

    dau= dau.repeat(64, axis=0).repeat(64, axis=1)

    asicno=setupczt(asicno)

    #plt.imshow(np.flipud(dau))
    #plt.imshow(asicno)

    # setting up dm numbering
    dm=np.linspace(1,16,16, dtype=np.int16)

    dm=np.reshape(dm,[4,4])



    dm= dm.repeat(16, axis=0).repeat(16, axis=1)

    print (dm[1::16,1::16])

    dmnumbers=rotatearound(dm)
    
    
    imax=asic_channel_numbers.shape[0]
    jmax=asic_channel_numbers.shape[1]

    xloc=np.zeros_like(dau)
    yloc=np.zeros_like(dau)
    dpuaddress=np.zeros_like(dau)
    dpuaddress[:,:]=0


    from bitstring import Bits
    print ('{:7s}  {:7s} {:8s}  {:8s} {:8s} {:8s}'.format('x-loc', 'y-loc', 'DAU' , 'DM' ,'CHANNEL' ,  'PIXEL', align='right'))
    for j in range(0, jmax):
        for i in range(0,imax):
            yloc[i,j]=(i-(imax+1)/2)*3
            xloc[i,j]=(j-(jmax+1)/2)*3

            dau_address   =Bits(uint=dau[i,j]-1      , length=2)
            asic_number   =Bits(uint=asicno[i,j]-1   , length=1)
            dm_address    =Bits(uint=dmnumbers[i,j]-1, length=4)
            asic_channel  =Bits(uint=asic_channel_numbers[i,j]-1     , length=7)

            dpu_address_bits=dau_address+asic_number+dm_address+asic_channel
            dpuaddress[i,j]=dpu_address_bits.uint

    import pandas as pd
    address_locations = { 
        'dpuaddress': np.ravel(dpuaddress),
        'xloc':np.ravel(xloc),
        'yloc':np.ravel(yloc),  
        'dau': np.ravel(dau) , 
        'dm' :  np.ravel(dmnumbers) , 
        'asic': np.ravel(asicno), 
        'channel':np.ravel(asic_channel_numbers) , 
        'pixel':np.ravel(pczt)}

    address_df=pd.DataFrame(address_locations, columns=[ 'dpuaddress', 'xloc', 'yloc','dau', 'dm', 'asic', 'channel', 'pixel' ] )

    import pprint

    result= address_df.sort_values(by=['dau', 'dm', 'asic', 'channel'], ascending=True)


    pp = pprint.PrettyPrinter(indent=4)
    pd.options.display.max_rows = 15

    pp.pprint (result) 


    #with open('gm_detector_locations.txt', 'wt') as out:
    #    pp = pprint.PrettyPrinter(indent=4, stream=out)
    #    pp.pprint(result)
    
## This function takes a DAU layout, clones it, rotates x 180 and returns a 2d layout for the entire MXGS CZT
def rotatearound(pdau3):
    pdau4=pdau3
    
    pdau1=np.rot90(pdau3,2)
    pdau2=pdau1 
    pdau12=np.concatenate( (pdau1, pdau2), axis=0)
    pdau34=np.concatenate( (pdau3, pdau4), axis=0)
    pczt=np.concatenate( (pdau12, pdau34), axis=1) 
    return pczt

#  function takes a 16x16  DM array and clones it 16 times to make a DAU, then roates to make a 128x128 CZT array
def setupczt(pixeldata):
    pchain1=np.concatenate((pixeldata, pixeldata, pixeldata,pixeldata)  , axis=1) 
    # should use numpy.tile 
    pdau3=np.concatenate((pchain1, pchain1, pchain1,pchain1)  , axis=0)    
    return rotatearound(pdau3)

def checkerboard_table(data, fmt='{:.0f}', bkg_colors=['white', 'white']):
    fig, ax = plt.subplots(figsize=(10,10))
    ax.set_axis_off()
    tb = Table(ax, bbox=[0,0,1,1])

    nrows, ncols = data.shape
    width, height = 1.0 / ncols, 1.0 / nrows

    # Add cells
    for (i,j), val in np.ndenumerate(data):
        # Index either the first or second item of bkg_colors based on
        # a checker board pattern
        idx = [j % 2, (j + 1) % 2][i % 2]
        color = bkg_colors[idx]

        tb.add_cell(i, j, width, height, text=fmt.format(val), 
                    loc='center', facecolor=color)

    # Row Labels...
    for i, label in enumerate(data.index):
        tb.add_cell(i, -1, width, height, text=label, loc='right', 
                    edgecolor='none', facecolor='none')
    # Column Labels...
    for j, label in enumerate(data.columns):
        tb.add_cell(-1, j, width, height/2, text=label, loc='center', 
                           edgecolor='none', facecolor='none')
    ax.add_table(tb)
    return fig

if __name__ == '__main__':
    main()

256
128
[[2 3]
 [1 4]]
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
x-loc    y-loc   DAU       DM       CHANNEL  PIXEL   
       dpuaddress  xloc  yloc  dau  dm  asic  channel  pixel
15929           0   -21   180    1   1     1        1    100
16184           1   -24   186    1   1     1        2    114
15928           2   -24   180    1   1     1        3    116
16057           3   -21   183    1   1     1        4     99
15802           4   -18   177    1   1     1        5     85
16058           5   -18   183    1   1     1        6     83
15801           6   -21   177    1   1     1        7    101
...           ...   ...   ...  ...  ..   ...      ...    ...
14969       16377   171   156    4  16     2      122    149
14714       16378   174   150    4  16     2      123    163
14968       16379   168   156    4  16     2      124    133
14585       16380   171   147    4  16     2      125    146
14842       16381   174   153    4  16     2      126    164
14840   